#### Magic Commands
Magic commands (those that start with `%`) are commands that modify a configuration of Jupyter Notebooks. A number of magic commands are available by default (see list [here](http://ipython.readthedocs.io/en/stable/interactive/magics.html))--and many more can be added with extensions. The magic command added in this section allows `matplotlib` to display our plots directly on the browser instead of having to save them on a local file.

In [1]:
%matplotlib inline

# Activity 8: Re-training a model dynamically
In this activity, we re-train our model every time new data is available.

First, we start by importing `cryptonic`. Cryptonic is a simple software application developed for this course that implements all the steps up to this section using Python classes and modules. Consider Cryptonic a template on how you could develop similar applications.

In [2]:
cd ../../cryptonic

/Users/lcapelo/Dropbox/book/cryptonic


In [ ]:
import pandas as pd

In [32]:
from tqdm import tqdm_notebook

In [ ]:
plt.style.use('seaborn-white')

In [4]:
from cryptonic import Model
from cryptonic import CoinMarketCap
import cryptonic.models.normalizations as normalizations

Using TensorFlow backend.
/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


### Fecthing Real-time Data
Throughout this project we have been using data originally provided by [CoinMarketCat](https://coinmarketcap.com/). We have created an interface for collecting both real-time and historical data as as part of `cryptonic`: the class `CoinMarketCap()`.

In [5]:
print(CoinMarketCap())



        Crypto-currency data comes from the website CoinMarketCap.
        CoinMarketCap is can be accessed at: https://coinmarketcap.com/

        The permission to use the data is available on their FAQ

            https://coinmarketcap.com/faq/

        and reads:

            "Q: Am I allowed to use content (screenshots, data, graphs, etc.) 
            for one of my personal projects and/or commercial use?

            R: Absolutely! Feel free to use any content as you see fit. 
            We kindly ask that you cite us as a source."
        
        


Our model is designed to work with daily data. Let's go ahead and collect historic daily data from CoinMarketCap (this is the same data used previously).

In [6]:
historic_data = CoinMarketCap.historic()

In [7]:
historic_data.head(3)

,date,open,high,low,close,volume,market_cap
0,2017-12-21,16642.4,17567.7,15342.7,15802.9,16516600000,278827000000
1,2017-12-20,17760.3,17934.7,16077.7,16624.6,22149700000,297526000000
2,2017-12-19,19118.3,19177.8,17275.4,17776.7,16894500000,320242000000


The data contains practically the same variables from our earlier dataset. However, much of the data comes from an earlier period. Recent Bitcoin prices have gained a lot of volatility if compared to the prices of a few years ago. Before using this data in our model, let's make sure to filter it to dates after January 1, 2017.

In [8]:
#
#  Using the Pandas API, filter the dataframe
#  for observations from 2017 only. 
# 
#  Hint: use the `date` column / variable.
#
model_data = 

### The `Model()` Class
We have also created the class `Model()` which compiles all the code we have written so far. We will use that class to build, train, and evaluate our model.

In [9]:
M = Model(data=model_data,
          variable='close',
          predicted_period_size=7)

In [10]:
M.build()

In [11]:
M.train(epochs=100, verbose=1)

Epoch 1/100
1/1 [==============================] - 1s 725ms/step - loss: 0.0115
Epoch 2/100
1/1 [==============================] - 0s 23ms/step - loss: 0.0100
Epoch 3/100
1/1 [==============================] - 0s 26ms/step - loss: 0.0091
Epoch 4/100
1/1 [==============================] - 0s 28ms/step - loss: 0.0083
Epoch 5/100
1/1 [==============================] - 0s 30ms/step - loss: 0.0077
Epoch 6/100
1/1 [==============================] - 0s 26ms/step - loss: 0.0072
Epoch 7/100
1/1 [==============================] - 0s 29ms/step - loss: 0.0068
Epoch 8/100
1/1 [==============================] - 0s 30ms/step - loss: 0.0064
Epoch 9/100
1/1 [==============================] - 0s 28ms/step - loss: 0.0060
Epoch 10/100
1/1 [==============================] - 0s 30ms/step - loss: 0.0056
Epoch 11/100
1/1 [==============================] - 0s 32ms/step - loss: 0.0053
Epoch 12/100
1/1 [==============================] - 0s 28ms/step - loss: 0.0050
Epoch 13/100
1/1 [==============================

We can now use the model for making predictions with the `predict()` method. The parameter `denormalized` will return values in the original scale of the data. In our case, US dollars.

In [12]:
M.predict(denormalized=True)

array([ 15785.41992188,  15281.00097656,  17791.39257812,  17695.06054688,
        17869.8203125 ,  17855.71875   ,  17691.33203125], dtype=float32)

We now evaluate our model to inspect the statistics for the last epoch of training compared to a single test week.

In [13]:
M.evaluate()

{'mape': 9.5899999999999999, 'mse': 0.0, 'rmse': 1713.98}

Finally, we can now save the trained model on disk for later use.

In [14]:
M.save('../lesson_4/activity_8/bitcoin_model_prod_v0.h5')

Our `Model()` class can also load a previously trained model when instantiated with the `path` parameter.

In [15]:
M = Model(path='../lesson_4/activity_8/bitcoin_model_prod_v0.h5',
          data=model_data,
          variable='close',
          predicted_period_size=7)

In [16]:
M.predict(denormalized=True)

array([ 15785.41992188,  15281.00097656,  17791.39257812,  17695.06054688,
        17869.8203125 ,  17855.71875   ,  17691.33203125], dtype=float32)

### New Data, Re-train Old Model
One strategy discussed earlier regards the re-training of our model with new data. In our case, our biggest concern is to shape data in a way that the model has been configured. As an example, we will configure our model to predict a week using 40 weeks. We will first train the model with the first 40 weeks of 2017, then continue to re-train it over the following weeks until we reach week 50.

In [17]:
print('Number of full weeks: {}'.format(len(model_data) // 7))

Number of full weeks: 50


First, let's build a model with the first set of data. Notice how we use `7*40 + 7` as the indexer. This is because we use 40 weeks for training and 1 week for testing. 

In [18]:
M = Model(data=model_data[0*7:7*40 + 7],
          variable='close',
          predicted_period_size=7)

In [19]:
M.build()

In [20]:
M.train()

In [33]:
#
#  Complete the range function and
#  the model_data filtering parameters
#  using an index to split the data in overlapping
#  groups of 7 days. Then, re-train our model
#  and collect the results.
#
#  The variables A, B, C, and D are placeholders.
#
results = []
for i in range(A, B):
    M.train(model_data[C:D])
    results.append(M.evaluate())

HBox(children=(IntProgress(value=0, max=9), HTML(value='')))

In [ ]:
M.predict(denormalized=True)

### New Data, New Model
Another strategy is to create and train a new model evey time new data is available. This approach tends to reduce catastrophic forgetting, but training time increases as data increases. 

It's implementation is quite simple.

Let's assume we have old data for 49 weeks of 2017 and after a week we now have new data. We represent this wtih the variables `old_data` and `new_data`. 

In [70]:
old_data = model_data[0*7:7*48 + 7]

In [71]:
new_data = model_data[0*7:7*49 + 7]

In [74]:
M = Model(data=old_data,
          variable='close',
          predicted_period_size=7)

In [75]:
M.build()
M.train()

In [76]:
M.predict(denormalized=True)

array([ 15775.79589844,  15595.73925781,  15938.69824219,  14605.47949219,
        14482.2265625 ,  14402.16894531,  14504.89648438], dtype=float32)

Now, assume that new data is available. Using this technicle we go ahead and create a new model using only the new data. 

In [77]:
#
#  Re-instantiate the model with the Model()
#  class using the new_data variable instead
#  of the old_data one. 
#

In [78]:
M.build()
M.train()

In [79]:
M.predict(denormalized=True)

array([ 15820.33691406,  15276.56640625,  17786.859375  ,  17703.76757812,
        17874.65429688,  17807.64257812,  17675.73828125], dtype=float32)

This approach is very simple to implement and tends to work well. We will be using this to deploy our application.